# Advent Of Code 2022

In [1]:
using System.IO;

## Day 1

In [2]:
IEnumerable<int> ReadCalories() {
    var total = 0;
    foreach(var line in File.ReadLines("../data/in-1.txt")) {
        if(int.TryParse(line, out var value)) {
            total += value;
        }
        else {
            yield return total;
            total = 0;
        }
    }
    yield return total;
}

Console.WriteLine(ReadCalories().Max());

69289


# Day 3

In [10]:
public class Day3 {

    int GetPriority(string str) 
    {
        long contains = 0;
        var half = str.Length / 2;
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i]);
            var mask = 1L << prio;
            contains |= mask;
        }
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i + half]);
            var mask = 1L << prio;
            if((contains & mask) > 1) {
                return prio;
            }
        }
        return 0;
    }
    
    int GetPriority(char c) 
    {
        if(c > 'Z') 
        {
            return (int)(c - 'a') + 1;
        }
        return (int)(c - 'A') + 27;
    }
    
    int Run3_1(string file) {
        return
            File
            .ReadLines(file)
            .Select(l => GetPriority(l))
            .Sum();
    }

    int GetPriority(string str1, string str2, string str3) 
    {
        var chars = str1.Distinct().Concat(str2.Distinct()).Concat(str3.Distinct());
        var c = 
            chars
            .GroupBy(c => c)
            .Single(g => g.Count() == 3)
            .Key;
        return GetPriority(c);
    }
    
    
    IEnumerable<int> Run3_2(string file) {
        var lines = File.ReadLines(file).ToList();
        for(var i=0; i<lines.Count; i+=3) {
            yield return GetPriority(lines[i], lines[i+1], lines[i+2]);
        }
    }
    

    public void Run() {
        Console.WriteLine(Run3_1("../data/in-3.txt"));
        Console.WriteLine(Run3_2("../data/in-3.txt").Sum());
    }
}

new Day3().Run();


7763
2569
